In [9]:
import os
import time
import datetime
import selenium.webdriver as webdriver
import chromedriver_binary
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'
PATH = 'C:\Program Files\Mozilla Firefox'
s=Service('C:\Program Files\Mozilla Firefox\geckodriver.exe')
driver = webdriver.Firefox(service=s)

In [11]:
symbol = ''
print('Enter the crypto ticker:')
symbol = input().upper()

Enter the crypto ticker:
eth


In [12]:
def xpath_element(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        element = []
    return element

In [13]:
def real_time_price(stock_code=symbol):
    url  = 'https://finance.yahoo.com/quote/'+stock_code+'?p='+stock_code+'&.tsrc=fin-srch'
    driver.get(url)
    ########## PRICE AND CHANGES ###########
    xpath = '//*[@id="quote-header-info"]/div[3]/div[1]/div[1]'
    stock_price_info = xpath_element(xpath)
#     print(stock_price_info.text.split()[0], stock_price_info.text.split()[1])
    if stock_price_info != []:
        stock_price_temp = stock_price_info.text.split()[0]
        if stock_price_temp.find('+') != -1:
            price = stock_price_temp.split('+')[0]
            try:
                change = '+' + stock_price_temp.split('+')[1] + ' ' + stock_price_info.text.split()[1]
            except IndexError:
                change = []
        elif stock_price_temp.find('-') != -1:
            price = stock_price_temp.split('-')[0]
            try:
                change = '-' + stock_price_temp.split('-')[1] + ' ' + stock_price_info.text.split()[1]
            except IndexError:
                change = []
        else:
            price,change = [],[]
    else:
        price, change = [],[]
    ########## Volume ###########
    xpath = '//*[@id="quote-summary"]/div[2]/table'
    volume_temp = xpath_element(xpath)
#     print(volume_temp.text.split()[-9])
    if volume_temp != []:
#         volume = volume_temp.text.split()[-9]
        for i, text in enumerate(volume_temp.text.split()):
            if text == 'Volume':
                volume = volume_temp.text.split()[i+1]
                break
            else:
                volume = []
    return [price, change, volume]

In [14]:
crypto_stonk = [symbol]

In [15]:
stonks = []
for i in (crypto_stonk):
    stonks.append(str(i) + '-USD')

In [16]:
while True:
    time_stamp = datetime.datetime.now()
    time_stamp = time_stamp.strftime('%y-%m-%d %H:%M:%S')
    testing = []
    for i in stonks:
        price, change, volume = real_time_price(i)
        for j in real_time_price(i):
            testing.append(j)
            trial = list(map(list,zip(*([iter(testing)]*3))))
            for k in trial:
                k.append(time_stamp)
    #         change the number if more parameters added
    df = pd.DataFrame(trial, columns=['Price','Change','Volume','Time'])
    df = df[['Time','Price','Change','Volume']]
    df = df.set_index('Time')
    df.to_csv(symbol + '_' +str(time_stamp[0:5]) + 'cryptodata.csv',mode='a', header=False)
    print(df)
    time.sleep(50)
driver.close()


                      Price            Change          Volume
Time                                                         
22-05-07 22:51:42  2,540.47  -134.99 (-5.05%)  16,492,146,688
                      Price            Change          Volume
Time                                                         
22-05-07 22:52:38  2,540.47  -134.99 (-5.05%)  16,492,146,688
                      Price            Change          Volume
Time                                                         
22-05-07 22:53:36  2,540.79  -134.32 (-5.02%)  16,499,248,128
                      Price            Change          Volume
Time                                                         
22-05-07 22:54:31  2,543.26  -132.01 (-4.93%)  16,537,401,344
                      Price            Change          Volume
Time                                                         
22-05-07 22:55:27  2,543.58  -131.59 (-4.92%)  16,549,725,184


KeyboardInterrupt: 

In [ ]:
# Future: Work on custom loop to set at every minute intervals in CST. 
# Future: Work on looping through different tickers 
# Future: Work on making these different tickers inside their individual excel_sheet
# Future: Inputs will be from most common gains from scraping Reddit's WSB